In [1]:
addprocs(12)  # parallel computing

12-element Array{Any,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13

In [2]:
require("demands_adjustment_Sioux_simp_gradi")

In [3]:
# jacob = zeros(length(demandsVec), numLinks)
# jacob = dzeros((length(demandsVec),numLinks), workers()[1:8], [8,1])

@time y = @parallel vcat for i=1:numODpairs
    solveJacob(i)
#     d = solveJacob(i)
#     println(d)
#     for j=1:numLinks
#         jacob[i,j] = d[j]
#     end
#     println("$(i)-th row completed")
end

y = reshape(y, numODpairs, numLinks)
println(y)

outfile = open("jacobian.json", "w")

JSON.print(outfile, y)

close(outfile)

# M = {1:length(demandsVec)}
# pmap(solveJacob, M)

elapsed time: 12.835634436 seconds (10849780 bytes allocated)
[-0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 1.0 -0.0 0.0 0.0 0.0 1.0 -0.0 0.0 0.0 0.0 1.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 -1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0
 -1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0
 -1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

In [4]:
# load Jacobian
jacobian = readall("jacobian.json");
jacobian = JSON.parse(jacobian);

jacob = zeros(numODpairs, numLinks)
for i = 1:numLinks
    for j = 1:numODpairs
        jacob[j, i] = jacobian[i][j]
    end
end

In [5]:
jacob

132x30 Array{Float64,2}:
 -0.0   0.0   0.0  0.0   0.0  -0.0  0.0  …   1.0  -0.0  0.0   0.0  0.0   1.0
  0.0   0.0   0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
  0.0  -1.0   0.0  0.0   0.0   0.0  0.0      1.0   0.0  0.0   1.0  0.0   0.0
 -1.0   0.0   0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0   0.0   1.0  0.0      0.0   1.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0   0.0   0.0  0.0  …   1.0   0.0  0.0   0.0  0.0   0.0
  0.0   0.0  -0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0  -0.0  0.0   0.0
 -1.0   0.0   1.0  0.0   0.0   0.0  0.0      0.0   1.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
 -1.0   0.0   0.0  0.0   0.0   0.0  0.0  …   0.0   0.0  0.0   0.0  0.0   1.0
  0.0   0.0   1.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0  -0.0   0.0  0.0     -0.0   

In [6]:
# compute the gradient
gradi_ = zeros(numODpairs)
for i = 1:numODpairs
    gradi_[i] = sum([2 * (x[j] - x_0[j]) * jacob[i, j] for j = 1:numLinks])
end

In [9]:
numNodes

12

In [10]:
OD_pair_label_dict = Dict()
OD_pair_label_dict_ = Dict()
label = 1
for i = 1:numNodes
    for j = 1:numNodes
        if i != j
            key = "($(i), $(j))"
            OD_pair_label_dict[key] = label
            OD_pair_label_dict_[label] = key
            label += 1
        end
    end
end

In [11]:
OD_pair_label_dict_[numODpairs]

"(12, 11)"

In [12]:
#load OD pair labels
odPairLabel = readall("od_pair_label_dict_Sioux_simp_refined.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__Sioux_simp_refined.json");
odPairLabel_ = JSON.parse(odPairLabel_);

In [13]:
# ensure the consistency of dimensions

gradi = zeros(numODpairs)

for i = 1:numODpairs
    if ~("($(odPairLabel_["$(i)"][1]), $(odPairLabel_["$(i)"][2]))" in keys(OD_pair_label_dict))
        gradi[i] = 0
    else
        gradi[i] = gradi_[OD_pair_label_dict["($(odPairLabel_["$(i)"][1]), $(odPairLabel_["$(i)"][2]))"]]
    end
end

In [17]:
length(demands)

144

In [20]:
odPairLabel_

Dict{UTF8String,Any} with 132 entries:
  "109" => {10,11}
  "29"  => {3,8}
  "81"  => {8,4}
  "54"  => {5,11}
  "105" => {10,6}
  "101" => {10,2}
  "32"  => {3,11}
  "30"  => {3,9}
  "2"   => {1,3}
  "1"   => {1,2}
  "74"  => {7,9}
  "41"  => {4,9}
  "78"  => {8,1}
  "65"  => {6,11}
  "51"  => {5,8}
  "53"  => {5,10}
  "119" => {11,9}
  "106" => {10,7}
  "27"  => {3,6}
  "75"  => {7,10}
  "42"  => {4,10}
  "33"  => {3,12}
  "28"  => {3,7}
  "50"  => {5,7}
  "52"  => {5,9}
  ⋮     => ⋮

In [21]:
# for convenience of computing the new demand vector
demandsVec = zeros(length(odPairLabel_))

for i = 1:length(demandsVec)
    demandsVec[i] = demands[(odPairLabel_["$i"][1], odPairLabel_["$i"][2])]
end

In [22]:
demandsVec

132-element Array{Float64,1}:
   90.0
   90.0
  450.0
  180.0
  270.0
  450.0
  720.0
  450.0
 1170.0
  450.0
  180.0
   90.0
   90.0
    ⋮  
 1260.0
  180.0
   90.0
  180.0
  540.0
  180.0
  180.0
  630.0
  540.0
  540.0
 1800.0
 1260.0

In [26]:
demands

Dict{(Int64,Int64),Float64} with 144 entries:
  (8,10)  => 1440.0
  (9,5)   => 720.0
  (3,6)   => 270.0
  (10,5)  => 900.0
  (6,9)   => 360.0
  (12,4)  => 540.0
  (4,4)   => 0.0
  (9,4)   => 630.0
  (10,11) => 3600.0
  (8,9)   => 720.0
  (9,1)   => 450.0
  (11,1)  => 450.0
  (8,11)  => 720.0
  (3,1)   => 90.0
  (1,10)  => 1170.0
  (1,12)  => 180.0
  (12,7)  => 630.0
  (4,5)   => 450.0
  (9,9)   => 0.0
  (11,8)  => 720.0
  (12,8)  => 540.0
  (2,4)   => 180.0
  (6,5)   => 180.0
  (4,9)   => 630.0
  (8,4)   => 630.0
  ⋮       => ⋮

In [51]:
# convert the demand data into dictionary; for convenience of solving the TAP problem and the inverse VI problem

demandsDict = Dict{(Int64,Int64), Float64}()

for key = [(int(split(key, ", ")[1][2:end]), int(split(key, ", ")[2][1:end-1])) for key = keys(odPairLabel)]
    demandsDict[key] = demandsVec[odPairLabel["($(key[1]), $(key[2]))"]]
end

In [53]:
demandsDict

Dict{(Int64,Int64),Float64} with 132 entries:
  (3,8)   => 180.0
  (9,5)   => 720.0
  (3,6)   => 270.0
  (10,5)  => 900.0
  (6,9)   => 360.0
  (12,4)  => 540.0
  (11,1)  => 450.0
  (9,4)   => 630.0
  (10,11) => 3600.0
  (8,9)   => 720.0
  (9,1)   => 450.0
  (8,11)  => 720.0
  (3,1)   => 90.0
  (1,10)  => 1170.0
  (1,12)  => 180.0
  (12,7)  => 630.0
  (4,5)   => 450.0
  (11,8)  => 720.0
  (12,8)  => 540.0
  (2,4)   => 180.0
  (6,5)   => 180.0
  (4,9)   => 630.0
  (8,4)   => 630.0
  (9,2)   => 180.0
  (1,2)   => 90.0
  ⋮       => ⋮

In [54]:
gradi

132-element Array{Float64,1}:
  -4212.86      
  -6880.0       
  -4296.04      
    460.0       
 -11983.8       
  -3396.07      
      0.0       
  -3845.07      
  -3479.45      
     -0.0       
  -3404.65      
  -3400.59      
      0.0       
      ⋮         
  -3621.27      
     -0.0       
  -3359.96      
     -0.0       
  -1919.41      
  -8198.69      
  -2096.07      
      2.46018e-9
  -2700.43      
  -8755.48      
  -4640.72      
  -6483.37      

In [55]:
# compute a descent direction

r = similar(gradi)
for i = 1:length(gradi)
    r[i] = -1 * gradi[i]
end

In [56]:
r

132-element Array{Float64,1}:
  4212.86      
  6880.0       
  4296.04      
  -460.0       
 11983.8       
  3396.07      
    -0.0       
  3845.07      
  3479.45      
     0.0       
  3404.65      
  3400.59      
    -0.0       
     ⋮         
  3621.27      
     0.0       
  3359.96      
     0.0       
  1919.41      
  8198.69      
  2096.07      
    -2.46018e-9
  2700.43      
  8755.48      
  4640.72      
  6483.37      

In [57]:
# compute a search direction

r_ = similar(r)
for i = 1:length(r)
    if (demandsVec[i] > 50) || (demandsVec[i] <= 50 && r[i] > 0)
        r_[i] = r[i]
    else
        r_[i] = 0
    end
end

In [58]:
r_

132-element Array{Float64,1}:
  4212.86      
  6880.0       
  4296.04      
  -460.0       
 11983.8       
  3396.07      
    -0.0       
  3845.07      
  3479.45      
     0.0       
  3404.65      
  3400.59      
    -0.0       
     ⋮         
  3621.27      
     0.0       
  3359.96      
     0.0       
  1919.41      
  8198.69      
  2096.07      
    -2.46018e-9
  2700.43      
  8755.48      
  4640.72      
  6483.37      

In [59]:
# line search

alphaList = Float64[]

for i = 1:length(r_)
    if r_[i] < 0
        push!(alphaList, -demandsVec[i]/r_[i])
    end
end

alpha_max = minimum(alphaList)

0.30527055337549003

In [60]:
length(r_)

132

In [61]:
# update

demandsVec_new = similar(demandsVec)
for i = 1:length(demandsVec)
    demandsVec_new[i] = demandsVec[i] + (alpha_max) * r_[i] 
end

In [62]:
demandsVec_new

132-element Array{Float64,1}:
 1376.06  
 2190.26  
 1761.45  
   39.5755
 3928.3   
 1486.72  
  720.0   
 1623.79  
 2232.17  
  450.0   
 1219.34  
 1128.1   
   90.0   
    ⋮     
 2365.47  
  180.0   
 1115.7   
  180.0   
 1125.94  
 2682.82  
  819.869 
  630.0   
 1364.36  
 3212.79  
 3216.67  
 3239.18  

In [63]:
demandsVec

132-element Array{Float64,1}:
   90.0
   90.0
  450.0
  180.0
  270.0
  450.0
  720.0
  450.0
 1170.0
  450.0
  180.0
   90.0
   90.0
    ⋮  
 1260.0
  180.0
   90.0
  180.0
  540.0
  180.0
  180.0
  630.0
  540.0
  540.0
 1800.0
 1260.0

In [66]:
demandsDict

Dict{(Int64,Int64),Float64} with 132 entries:
  (3,8)   => 180.0
  (9,5)   => 720.0
  (3,6)   => 270.0
  (10,5)  => 900.0
  (6,9)   => 360.0
  (12,4)  => 540.0
  (11,1)  => 450.0
  (9,4)   => 630.0
  (10,11) => 3600.0
  (8,9)   => 720.0
  (9,1)   => 450.0
  (8,11)  => 720.0
  (3,1)   => 90.0
  (1,10)  => 1170.0
  (1,12)  => 180.0
  (12,7)  => 630.0
  (4,5)   => 450.0
  (11,8)  => 720.0
  (12,8)  => 540.0
  (2,4)   => 180.0
  (6,5)   => 180.0
  (4,9)   => 630.0
  (8,4)   => 630.0
  (9,2)   => 180.0
  (1,2)   => 90.0
  ⋮       => ⋮